<a href="https://colab.research.google.com/github/lourish789/Quizlet/blob/main/Quiz_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [33]:
import requests
import json

def fetch_quiz_data(amount=10, difficulty="medium", category=9, type="multiple"):
    url = f"https://opentdb.com/api.php?amount={amount}&difficulty={difficulty}&category={category}&type={type}"
    response = requests.get(url)
    data = response.json()
    return data['results'] if data['response_code'] == 0 else []

quiz_data = fetch_quiz_data()
print(json.dumps(quiz_data, indent=4))  # View the raw data

[
    {
        "type": "multiple",
        "difficulty": "medium",
        "category": "General Knowledge",
        "question": "Which American manufactured submachine gun was informally known by the American soldiers that used it as &quot;Grease Gun&quot;?",
        "correct_answer": "M3",
        "incorrect_answers": [
            "Colt 9mm",
            "Thompson",
            "MAC-10"
        ]
    },
    {
        "type": "multiple",
        "difficulty": "medium",
        "category": "General Knowledge",
        "question": "The architect known as Le Corbusier was an important figure in what style of architecture?",
        "correct_answer": "Modernism",
        "incorrect_answers": [
            "Neoclassical",
            "Baroque",
            "Gothic Revival"
        ]
    },
    {
        "type": "multiple",
        "difficulty": "medium",
        "category": "General Knowledge",
        "question": "What character was once considered to be the 27th letter of the alphabet?"

In [34]:
from datasets import Dataset
import pandas as pd

# Convert OpenTDB data to a format suitable for training
def prepare_data(quiz_data):
    formatted_data = []
    for item in quiz_data:
        formatted_data.append({
            "input_text": f"Generate a {item['difficulty']} question about {item['category']}",
            "target_text": item["question"]
        })
    return formatted_data

formatted_data = prepare_data(quiz_data)

# Convert to Hugging Face dataset
df = pd.DataFrame(formatted_data)
dataset = Dataset.from_pandas(df)

In [36]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def tokenize_function(example):
    # Tokenize both input and target text
    inputs = tokenizer(example["input_text"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    targets = tokenizer(example["target_text"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")

    # Prepare the input for the model
    input_ids = inputs.input_ids.squeeze()  # Remove batch dimension if it's 1
    attention_mask = inputs.attention_mask.squeeze()
    labels = targets.input_ids.squeeze()  # Use target text as labels

    # Replace padding token IDs with -100 so they are ignored in loss calculation
    labels[labels == tokenizer.pad_token_id] = -100

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,  # Provide labels for the decoder
    }

tokenized_dataset = dataset.map(tokenize_function, batched=False, remove_columns=['input_text', 'target_text']) # batched=False, remove_columns

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


TrainOutput(global_step=6, training_loss=6.294822057088216, metrics={'train_runtime': 5.16, 'train_samples_per_second': 5.814, 'train_steps_per_second': 1.163, 'total_flos': 4060254044160.0, 'train_loss': 6.294822057088216, 'epoch': 3.0})